In [1]:
! pip install transformers

     |████████████████████████████████| 2.0MB 17.8MB/s 
     |████████████████████████████████| 870kB 50.5MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=3c21d956daa1434df20e876c7cc05d2a08cae90767b2d9f5c54fa96ed7ac775d
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
cd drive/MyDrive/LXPER

/content/drive/MyDrive/LXPER


In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter="\t", header=None)[:2000]
test_data = pd.read_csv("./cola_public/raw/in_domain_dev.tsv", delimiter="\t", header=None)[:2000]

In [4]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
use_cuda = True

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
if use_cuda and torch.cuda.is_available():
    model.cuda()
model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [5]:
from transformers import AdamW
import torch.nn as nn
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_batch = list(data.iloc[:,3])
labels = list(data.iloc[:,1])


encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
print(input_ids.shape)
print(attention_mask.shape)


torch.Size([2000, 47])
torch.Size([2000, 47])


In [7]:
encoding['labels'] = torch.tensor(labels)

In [8]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [9]:
device = 'cuda:0'
model = model.to(device)
encoding = encoding.to(device)

In [10]:
for epoch in range(3):
    for i in range(data.shape[0]):
        input = encoding['input_ids'][i].view(1, -1)
        mask = encoding['attention_mask'][i].view(1, -1)
        label = encoding['labels'][i].view(-1)

        optimizer.zero_grad()

        output = model(input, mask, labels=label)
        loss = criterion(output.logits, label)
        print(loss)
        print('EPOCH:',epoch, '%:', round(i/data.shape[0], 4))
        loss.backward()
        optimizer.step()

Streaming output truncated to the last 5000 lines.
tensor(0.8989, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.75
tensor(0.9032, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7505
tensor(0.0723, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.751
tensor(0.2094, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7515
tensor(0.3164, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.752
tensor(0.1374, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7525
tensor(0.1644, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.753
tensor(0.0527, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7535
tensor(0.0799, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.754
tensor(0.0528, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7545
tensor(2.1047, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.755
tensor(0.2305, device='cuda:0', grad_fn=<NllLossBackward>)
EPOCH: 1 %: 0.7555
tensor(0.0596, devic

In [11]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
test_text = list(test_data.iloc[:,3])
test_label = list(test_data.iloc[:,1])


In [13]:
test_encoding = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True)

test_encoding = test_encoding.to(device)
test_input_ids = test_encoding['input_ids']
test_attention_mask = test_encoding['attention_mask']

In [14]:
pred = []
for i in range(test_data.shape[0]):
        input = test_encoding['input_ids'][i].view(1, -1)
        mask = test_encoding['attention_mask'][i].view(1, -1)
        
        output = model(input, mask)
        output = int(output.logits.argmax())
        pred.append(output)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(test_label, pred)

0.8064516129032258

In [16]:
ls

BART.ipynb  BERT.ipynb  cola_public/  GPT1.ipynb  models/


In [17]:

torch.save(model.state_dict(), './models/bert_statedict')


In [ ]:
test_encoding = tokenizer(["I are a boy"], return_tensors='pt', padding=True, truncation=True)
test_encoding = test_encoding.to(device)
test_input_ids = test_encoding['input_ids']
test_attention_mask = test_encoding['attention_mask']

In [ ]:
print(torch.nn.functional.softmax(model(test_input_ids, test_attention_mask).logits, dim=1)[0][1])

tensor([0.9513, 0.0487], device='cuda:0', grad_fn=<SelectBackward>)
